In [6]:
from elasticsearch import Elasticsearch as es

In [7]:
search = es('http://localhost:9200')
clientInfo = search.info()
print(clientInfo.body)

{'name': '5214b71eb6ae', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'tJ5uYxiXQcC9LXBGMelG7Q', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [8]:
search.indices.delete(index='my_index', ignore_unavailable=True)
search.indices.create(index="my_index", settings={"index":{"number_of_replicas":1,"number_of_shards":3}}) # Replica is the copies of data, shards is the data segments

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [9]:
document = {
    'title':'title',
    'text':'text',
    'created on':'18-01-2025'
}
response = search.index(index='my_index', body=document)
response

ObjectApiResponse({'_index': 'my_index', '_id': 'OoNS1JQBYxtjn3IflrPy', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [10]:
response['result']

'created'

In [19]:
%pip install tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [20]:
import json
from tqdm import tqdm

In [12]:
dummyData = json.load(open("dummy_data.json"))
print(dummyData)

[{'title': 'Sample Title 1', 'text': 'This is the first sample document text.', 'created_on': '2024-09-22'}, {'title': 'Sample Title 2', 'text': 'Here is another example of a document.', 'created_on': '2024-09-24'}, {'title': 'Sample Title 3', 'text': 'The content of the third document goes here.', 'created_on': '2024-09-24'}]


In [13]:
def insertDocument(data):
    response = search.index(index='my_index',document=data)
    return response

def printResponse(response):
    print(f"""Document_ID:{response["_id"]} has been {response["result"]}, which has {response["_shards"]["total"]} shards""")

for doc in dummyData:
    res = insertDocument(doc)
    printResponse(res)

Document_ID:O4Na1JQBYxtjn3IfPLOb has been created, which has 2 shards
Document_ID:PINa1JQBYxtjn3IfPbME has been created, which has 2 shards
Document_ID:PYNa1JQBYxtjn3IfPbMb has been created, which has 2 shards


In [14]:
from pprint import pprint

In [15]:
indexMap = search.indices.get_mapping(index='my_index')
pprint(indexMap["my_index"]["mappings"]["properties"])

{'created on': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
                'type': 'text'},
 'created_on': {'type': 'date'},
 'text': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
          'type': 'text'},
 'title': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
           'type': 'text'}}


In [17]:
rec = search.delete(index="my_index", id = "O4Na1JQBYxtjn3IfPLOb")
print(rec)

NotFoundError: NotFoundError(404, "{'_index': 'my_index', '_id': 'O4Na1JQBYxtjn3IfPLOb', '_version': 3, 'result': 'not_found', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}")

In [18]:
search.indices.delete(index="my_index", ignore_unavailable= True)
search.indices.create(index = "my_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [21]:
dummyData = json.load(open("dummy_data.json"))
#print(dummyData)

documentId = []
for document in tqdm(dummyData, total=len(dummyData)):
    response = search.index(index = "my_index", document=document)
    documentId.append(response['_id'])


100%|██████████| 3/3 [00:00<00:00, 24.75it/s]


In [22]:
documentId

['PoPU2ZQBYxtjn3IfVrPz', 'P4PU2ZQBYxtjn3IfV7NK', 'QIPU2ZQBYxtjn3IfV7Nb']

In [28]:
pprint(search.get(index='my_index', id=documentId[0]).body)

{'_id': 'PoPU2ZQBYxtjn3IfVrPz',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'created_on': '2024-09-22',
             'text': 'This is the first sample document text.',
             'title': 'Sample Title 1'},
 '_version': 1,
 'found': True}


In [27]:
pprint(search.get(index='my_index', id=documentId[0]).body['_source'])

{'created_on': '2024-09-22',
 'text': 'This is the first sample document text.',
 'title': 'Sample Title 1'}


In [26]:
search.get(index='my_index',id=documentId[0]).body

{'_index': 'my_index',
 '_id': 'PoPU2ZQBYxtjn3IfVrPz',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 'found': True,
 '_source': {'title': 'Sample Title 1',
  'text': 'This is the first sample document text.',
  'created_on': '2024-09-22'}}

Count method

In [29]:
response = search.count(index='my_index')
pprint(response.body)


{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 3}


In [31]:
query = {
    "range":{
        "created_on":{
            "gte":"2024-09-24",
            "lte":"2024-09-24",
            "format":"yyyy-MM-dd"
        }
    }
}
response = search.count(index='my_index',query=query)
pprint(response.body)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 2}


In [37]:
response = search.indices.exists(index='my_index')
pprint(response.body)

True


In [40]:
response = search.exists(index='my_index',id=documentId[0])
pprint(response.body)

True


Update method

In [44]:
response = search.update(index='my_index', id = documentId[0],
                         script={
                             "source":"ctx._source.title = params.title",
                             "params":{
                                 "title":"New Title"
                             }
                         },
                         )
pprint(response.body)

{'_id': 'PoPU2ZQBYxtjn3IfVrPz',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 3,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 2,
 'result': 'updated'}


In [60]:
response = search.get(index='my_index', id=documentId[0])
pprint(response.body)

{'_id': 'PoPU2ZQBYxtjn3IfVrPz',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 6,
 '_source': {'created_on': '2024-09-22',
             'new_field': 'new value',
             'text': 'This is the first sample document text.',
             'title': 'New Title'},
 '_version': 5,
 'found': True}


In [48]:
response = search.update(index='my_index', id=documentId[0],
                         script={
                             "source":"ctx._source.new_field = 'new value'"
                         })
pprint(response.body)

{'_id': 'PoPU2ZQBYxtjn3IfVrPz',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 4,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 3,
 'result': 'updated'}


In [55]:
response = search.update(index='my_index', id=documentId[0],
                         doc={"new field 2": "values"})

pprint(response.body)

{'_id': 'PoPU2ZQBYxtjn3IfVrPz',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 5,
 '_shards': {'failed': 0, 'successful': 0, 'total': 0},
 '_version': 4,
 'result': 'noop'}


In [59]:
response = search.update(index='my_index', id = documentId[0],
                         script={
                             "source" : "ctx._source.remove('new field 2')"
                         })

pprint(response.body)

{'_id': 'PoPU2ZQBYxtjn3IfVrPz',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 6,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 5,
 'result': 'updated'}


In [61]:
response = search.update(index='my_index', id = '1',
                         doc = {
                             "book_title": "Life is a race",
                             "book_id": "1234"
                         },
                         doc_as_upsert=True)

pprint(response.body)

{'_id': '1',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 7,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


In [63]:
response = search.count(index='my_index')
response

ObjectApiResponse({'count': 4, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})